# Notes for later reflection

This is an analysis of the gcdist function Daryl wrote in the Utility.jl file. I have modified it based on this analysis to fix the problem we were seeing, which was that occasionally it did not work. Although the error was a small percentage (***very small*** for large n) it was still a curiosity worth investigating. The error was sourced in the acos argument as occasionally that argument would be slightly more than one. This analysis confirms the value is **very** close to 1, and is probably just an overflow error. Based on this I changed the gcdist function by inserting a try statement and returning zero if it makes it to the catch statement, so running this code now will not reproduce the results below as the function is "fixed" now.

In [29]:
using DataFrames, CSV, DelimitedFiles, Plots, FFTW
include("/home/kirk/Documents/research/CHANDRATest/DJM.jl")

Main.DJM

In [30]:
using Formatting
function testGCDist(powList)
    justPow=powList.power[:]
    rdistError=[]
    for i in 1:length(justPow)
        print(format(i/length(justPow)*100,precision=2),"% complete\r") #output % tracker
        for j in (i+1):length(justPow) #i+1 accounts for indistinguishability
            push!(rdistError, DJM.gcdist(powList.ra[j],powList.dec[j],powList.ra[i],powList.dec[i]))
        end
    end
    return rdistError
end

testGCDist (generic function with 1 method)

In [24]:
pows12=CSV.read("cxc.fftinfo.all12")
pows8=CSV.read("cxc.fftinfo.all8")

,obsid,srcnum,startdate,exptime,ra,dec,freq,power
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1026,26,51833.8,6458.48,19.9382,-73.4501,0.00123975,13.2664
2,1026,8,51833.8,6458.48,19.5522,-73.6193,0.154427,9.16655
3,1027,107,51855.0,6133.27,18.8667,-73.3961,0.0185454,9.14431
4,1027,107,51855.0,6133.27,18.8667,-73.3961,0.144096,8.02378
5,1027,36,51855.0,6133.27,19.782,-73.5006,0.0813522,8.00401
6,1027,40,51855.0,6133.27,19.7033,-73.4917,0.0417401,8.04439
7,1027,54,51855.0,6133.27,19.5056,-73.4696,0.14484,8.14304
8,1027,58,51855.0,6133.27,19.4489,-73.4632,0.0598763,8.05626
9,1027,91,51855.0,6133.27,19.0827,-73.4213,0.0622972,8.75528


In [31]:
rdist12error=testGCDist(pows12)
rdist8error=testGCDist(pows8)

63331885-element Array{Any,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [34]:
rdist12errorVals=[]
for i in 1:length(rdist12error)
    if rdist12error[i]!=0
        push!(rdist12errorVals,rdist12error[i])
    end
end

In [43]:
println(mean(rdist12errorVals)," mean of 12 list error")
println(std(rdist12errorVals)," std of 12 list error")

1.0000000000000002 mean of 12 list error
0.0 std of 12 list error


In [44]:
rdist8errorVals=[]
for i in 1:length(rdist8error)
    if rdist8error[i]!=0
        push!(rdist8errorVals,rdist8error[i])
    end
end

In [45]:
println(mean(rdist8errorVals)," mean of 8 list error")
println(std(rdist8errorVals)," std of 8 list error")

1.0000000000000002 mean of 8 list error
0.0 std of 8 list error
